<a href="https://colab.research.google.com/github/reypapin/Generador-de-Dominios/blob/main/Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

# Upload the file trained_model_checkpoint.h5 from your local directory
uploaded = files.upload()


Saving trained_model_checkpoint.h5 to trained_model_checkpoint (1).h5


In [6]:
import numpy as np
import string
from tensorflow.keras.models import load_model

# Load the model from the file
model = load_model('trained_model_checkpoint.h5')

# Seed domain for generating text
seed_domain = "google"
padding_length = 15 - len(seed_domain)
sentence = seed_domain + " " * padding_length

generated = ""
maxlen=15

# Define the alphabet
alphabet = string.ascii_lowercase + "0123456789-._ "
print(alphabet)

# Get unique characters from the alphabet
chars = sorted(list(set(alphabet)))
print("Total chars:", len(chars))

# Create dictionaries for character to index and index to character mappings
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Define the sampling function
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

if len(sentence) < 15:
    sentence = sentence.ljust(15)

for diversity in [0.5]:
    print("...Diversity:", diversity)

print("...Generated: ", generated)
print("...Diversity:", diversity)

# Generate domain names
for i in range(30000):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    sentence = sentence[1:] + next_char
    generated += next_char

    if i % 10000 == 0:  # Check if i is a multiple of 1000
        print("Value of i:", i)

print("...Generated: ", generated)
print()

abcdefghijklmnopqrstuvwxyz0123456789-._ 
Total chars: 40
...Diversity: 0.5
...Generated:  
...Diversity: 0.5


<ipython-input-6-2d23e7094501>:31: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


Value of i: 0
Value of i: 10000
Value of i: 20000
...Generated:   oopaos            eola e              ainardhmr       o h              ailis o              aogeieoe        a r              anpasi          m s              uaau a              eeeisga e              eoita            datol            aoeit i              emgrau v   .              oesaseeih d  . o              aelos            tuii -          i a              eaiiae i              aat a      _              iadssertiee       m s              oepingliye     u a              aassboie e              eiiens           c r              a-paran         c s              eaiian          c s              eakes i              aetso t              enmcsr             oeitrs          e e              liseew          e e              xiaea o              ahcmd             aaerr r              eoosn t              oseaec            l t              nawisa             aosaaaeb             amieoe r              palisao         deriak      

In [9]:
import pandas as pd

# Length of each segment
segment_length = 15

# Calculate the number of segments
num_segments = len(generated) // segment_length

# Create a list to store the segments
segments = [generated[i:i + segment_length] for i in range(0, len(generated), segment_length)]

# Create the "Test" DataFrame
df2 = pd.DataFrame({"Segment": segments})



In [10]:
import pandas as pd

def fix_domain(segment):
    # Remove leading spaces
    fixed_segment = segment.strip()

    # Find the first space after encountering letters and cut there
    for i, char in enumerate(fixed_segment):
        if char.isspace() and any(c.isalpha() for c in fixed_segment[:i]):
            fixed_segment = fixed_segment[:i]
            break

    return fixed_segment

# Concatenate all characters of each row without spaces
df2["Joined_Segment"] = df2["Segment"].str.replace(" ", "")

# Apply the fix_domain function to the "Joined_Segment" column
df2["Fixed_Segment"] = df2["Joined_Segment"].apply(fix_domain)

# Drop the "Joined_Segment" column if no longer needed
df2.drop(columns=["Joined_Segment"], inplace=True)
df2.drop(columns=["Segment"], inplace=True)


In [12]:
print(df2)

     Fixed_Segment
0           oopaos
1            eolae
2           ainard
3            hmroh
4              ail
...            ...
1995           eis
1996        losoah
1997           afe
1998             o
1999          aees

[2000 rows x 1 columns]


In [14]:
uploaded = files.upload()

Saving tranco_6JVNX.csv.gz to tranco_6JVNX.csv (1).gz


In [18]:
import pandas as pd
df = pd.read_csv('tranco_6JVNX.csv.gz', compression='gzip')
df = df.drop('1', axis=1)
df = df.rename(columns={'google.com': 'Domain'})
df.loc[df.index.max() + 1] = 'google.com'
print(df)


                        Domain
0                 a-msedge.net
1                  youtube.com
2                 facebook.com
3                microsoft.com
4                amazonaws.com
...                        ...
3576071            lmd-inc.com
3576072               yoow.com
3576073               ckie.com
3576074  hk-stanley-market.com
3576075             google.com

[3576076 rows x 1 columns]


In [20]:
!pip install tldextract
!pip install wordninja

import pandas as pd
import tldextract
import wordninja

# Function to split a domain into meaningful keywords
def split_domain_into_keywords(domain):
    # Use tldextract to extract parts of the domain
    extracted = tldextract.extract(domain)

    # Get separated parts of the domain
    #subdomain = extracted.subdomain
    domain_name = extracted.domain
    tld = extracted.suffix

    # Split the domain into keywords using wordninja
    #subdomain_keywords = wordninja.split(subdomain) if subdomain else []
    #domain_keywords = wordninja.split(domain_name)

    return {
        #"Subdomain Keywords": subdomain_keywords,
        #"Domain Keywords": domain_keywords,
        "TLD": tld,
        "Domain Name": domain_name
    }

batch_size = 500000
results = []

for batch_number, start in enumerate(range(0, len(df), batch_size), start=1):
    end = start + batch_size
    batch_df = df.iloc[start:end]

    print(f"Processing batch {batch_number}")

    # Apply the function to the 'Domain' column and create additional columns
    batch_df[['TLD', 'Domain Name']] = batch_df['Domain'].apply(split_domain_into_keywords).apply(pd.Series)

    results.append(batch_df)

# Concatenate the results into a new DataFrame
df = pd.concat(results)

# Filter the dataframe to keep only rows where the length of the element in the "Domain Name" column is less than or equal to 15
df = df[df['Domain Name'].str.len() <= 15]

# Reset the dataframe index to maintain row order
df.reset_index(drop=True, inplace=True)

# Print the resulting DataFrame
print(df)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=64fed343455de104aed515987c68264ae4b3dae81dbdecb570fa3e2796c4ca1a
  Stored in directory: /root/.cache/pip/wheels/aa/44/3a/f2a5c1859b8b541ded969b4cd12d0a58897f12408f4f51e084
Successfully built wordninja
Processing batch 1


<ipython-input-20-360fd6cfa58a>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df[['TLD', 'Domain Name']] = batch_df['Domain'].apply(split_domain_into_keywords).apply(pd.Series)
<ipython-input-20-360fd6cfa58a>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df[['TLD', 'Domain Name']] = batch_df['Domain'].apply(split_domain_into_keywords).apply(pd.Series)


Processing batch 2


<ipython-input-20-360fd6cfa58a>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df[['TLD', 'Domain Name']] = batch_df['Domain'].apply(split_domain_into_keywords).apply(pd.Series)
<ipython-input-20-360fd6cfa58a>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df[['TLD', 'Domain Name']] = batch_df['Domain'].apply(split_domain_into_keywords).apply(pd.Series)


Processing batch 3


...

Processing batch 7


<ipython-input-20-360fd6cfa58a>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df[['TLD', 'Domain Name']] = batch_df['Domain'].apply(split_domain_into_keywords).apply(pd.Series)
<ipython-input-20-360fd6cfa58a>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df[['TLD', 'Domain Name']] = batch_df['Domain'].apply(split_domain_into_keywords).apply(pd.Series)


Processing batch 8


<ipython-input-20-360fd6cfa58a>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df[['TLD', 'Domain Name']] = batch_df['Domain'].apply(split_domain_into_keywords).apply(pd.Series)
<ipython-input-20-360fd6cfa58a>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df[['TLD', 'Domain Name']] = batch_df['Domain'].apply(split_domain_into_keywords).apply(pd.Series)


                 Domain  TLD  Domain Name
0          a-msedge.net  net     a-msedge
1           youtube.com  com      youtube
2          facebook.com  com     facebook
3         microsoft.com  com    microsoft
4         amazonaws.com  com    amazonaws
...                 ...  ...          ...
3039137  black-latte.eu   eu  black-latte
3039138     lmd-inc.com  com      lmd-inc
3039139        yoow.com  com         yoow
3039140        ckie.com  com         ckie
3039141      google.com  com       google

[3039142 rows x 3 columns]


In [21]:
import pandas as pd

# Assuming you already have your dataframe df loaded

# Calculate the frequency of each TLD
tld_frequency = df['TLD'].value_counts()

# Select the top 50 most common TLDs
top_50_tld = tld_frequency.head(50)

# Convert the result into a list
TLD = top_50_tld.index.tolist()

# Now TLD contains the top 50 most common TLDs
print(TLD)


['com', 'ru', 'net', 'org', 'de', 'co.uk', 'top', 'com.br', 'info', 'cn', 'nl', 'xyz', 'fr', 'com.au', 'ca', 'it', 'jp', 'io', 'pl', 'co', 'ch', 'dk', 'co.jp', 'eu', 'biz', 'in', 'us', 'ir', 'online', 'se', 'es', 'be', 'cz', 'site', 'at', 'cc', 'club', 'ro', 'com.cn', 'buzz', 'pro', 'com.tr', 'no', 'co.kr', 'fi', 'co.nz', 'tv', 'hu', 'me', 'shop']


In [22]:
import pandas as pd
import numpy as np

# Assuming you already have your dataframe df2 loaded and you have the variable TLD

# Randomly choose a TLD from the TLD list for each domain in Fixed_Segment
random_tld = np.random.choice(TLD, size=len(df2))

# Concatenate the random TLD to the domain name
df2['DGA'] = df2['Fixed_Segment'] + '.' + random_tld
df2.drop(columns=["Fixed_Segment"], inplace=True)

# Show the first few rows of the resulting dataframe
print(df2)


                DGA
0        oopaos.xyz
1      eolae.com.br
2         ainard.jp
3          hmroh.ch
4            ail.ir
...             ...
1995         eis.tv
1996  losoah.com.cn
1997        afe.top
1998           o.cc
1999       aees.top

[2000 rows x 1 columns]
